# ANÁLISIS EXPLORATORIO DE DATOS

## 1. IMPORTAMOS PAQUETES Y DATOS

In [1]:
import pandas as pd
import zipfile
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%config IPCompleter.greedy = True
import warnings
warnings.filterwarnings('ignore')

Ruta archivo de calidad de datos

In [2]:
ruta_principal = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/EVOLVE/Data Science/EVOLVE/Fernando_Costa/Practicas/Mini_Proyecto_EDA/'
carpeta = '002_archivos/'

Nombres de los ficheros de datos.

In [3]:
df_calidad = 'trabajo_resultado_calidad.pickle'

Cargamos los datasets

In [4]:
df = pd.read_pickle(ruta_principal + carpeta + df_calidad)

## 2. CONCLUSIONES Y CAMBIOS NECESARIOS

Observamos lo siguiente sobre las variables categóricas y numéricas: La norma es dejar lo nulos como NaN 
- En ambos casos hay variables con muchos nulos pero tiene sentido que sean nulos porque son datos que no era necesario rellenarlos
    - crm_cd_4, crm_cd_3 y crm_cd_2 son los códigos de los crímenes secundarios. Entiendo que no hubo crímenes secundarios
    - weapon_used_cd y weapon_desc: Habla de las armas usadas. Entiendo que no hubo armas
    - cross_street: Identifica si el crimen se cometió en un cruce. Entiendo que no fue en un cruce
    - mocodes: Habla del modus operandi específico del crimen. Entiendo que no encontraron un patrón identificable

- En el caso de vict_descent vamos a borrar los 2 registros que hay con '-'
- En el caso de vict_sex, aparecen registros con ['M', 'F', nan, 'X', 'H', '-']. Vamos a dejar solamente M: Male, F: Female y U: Unknown y borraremos el registro con '-'
- premis_cd y premis_desc contienen información sobre las instalaciones donde se produjo el crimen (código y descripción). Desconocemos el significado de los códigos, pero observamos que los registros de (df.premis_cd.notna())&(df.premis_desc.isna()) se concentran en unos cuantos códigos y habría que investigarlos con una persona de dentro de la organización para obtener mas información. De momento 
    - premis_cd podrían ser que no lo hayan escrito o que no tengan códogo para ese sitio concreto y habría que investigarlo con alguien de dentro
    - premis_desc que son nulos son los mismos que premis_cd por lo que los dejemos como nulos
- En el caso de status, los vamos a borrar porque no hay información en casi ninguna variable y solamente es un caso 
- Por último, crm_cd_1. Este es el código del delito principal que siempre debe ser un código mas bajo (mas bajo = mas grave) que los códigos de crm_cd_1, 2 3 y 4. Así que vamos a ponerles el mismo código por practicar entendiendo que se equivocaron al rellenar el campo y pusieron la información en el crm_cd_2 en vez de el crm_cd_1

In [5]:
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331240
crm_cd_3          330513
crm_cd_2          308186
cross_street      280379
weapon_desc       222974
weapon_used_cd    222974
mocodes            49840
vict_descent       47578
vict_sex           47572
premis_desc          201
premis_cd              5
crm_cd_1               3
crm_cd_desc            0
vict_age               0
part_1_2               0
area_name              0
rpt_dist_no            0
area                   0
time_occ               0
date_rptd              0
date_occ               0
crm_cd                 0
status                 0
status_desc            0
location               0
lat                    0
lon                    0
dtype: int64

### VARIABLES

#### MOCODES

Imputamos los nulos de 'mocodes' por 9999 para tenerlos identificados pero que sea un valor numérico de 4 dígitos

In [6]:
var_imputar_numero = ['mocodes']
numero = 9999
df[var_imputar_numero] = df[var_imputar_numero].fillna(numero)
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331240
crm_cd_3          330513
crm_cd_2          308186
cross_street      280379
weapon_desc       222974
weapon_used_cd    222974
vict_descent       47578
vict_sex           47572
premis_desc          201
premis_cd              5
crm_cd_1               3
vict_age               0
mocodes                0
crm_cd_desc            0
crm_cd                 0
part_1_2               0
rpt_dist_no            0
area_name              0
area                   0
time_occ               0
date_occ               0
date_rptd              0
status                 0
status_desc            0
location               0
lat                    0
lon                    0
dtype: int64

#### STATUS

Borramos el registro de status

In [7]:
df.loc[(df.status.isna())]

,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,mocodes,...,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
dr_no,,,,,,,,,,,,,,,,,,,,,


In [8]:
df = df.loc[df.status.notna()]
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331240
crm_cd_3          330513
crm_cd_2          308186
cross_street      280379
weapon_desc       222974
weapon_used_cd    222974
vict_descent       47578
vict_sex           47572
premis_desc          201
premis_cd              5
crm_cd_1               3
vict_age               0
mocodes                0
crm_cd_desc            0
crm_cd                 0
part_1_2               0
rpt_dist_no            0
area_name              0
area                   0
time_occ               0
date_occ               0
date_rptd              0
status                 0
status_desc            0
location               0
lat                    0
lon                    0
dtype: int64

#### VICT_DESCENT

Borramos los dos registros con '-' e imputamos los registros de vict_descent según la distribución de los datos 

In [9]:
df = df.loc[~(df.vict_descent=='-')]

#### CRM_CD_1

Imputamos los valores de crm_cd_1 que son nulos por los de crm_cd_2

In [10]:
df['crm_cd_1'] = df['crm_cd_1'].fillna(df['crm_cd_2'])
df.isna().sum().sort_values(ascending=False)

crm_cd_4          331239
crm_cd_3          330512
crm_cd_2          308185
cross_street      280378
weapon_desc       222973
weapon_used_cd    222973
vict_descent       47578
vict_sex           47572
premis_desc          201
premis_cd              5
date_rptd              0
vict_age               0
mocodes                0
crm_cd_desc            0
crm_cd                 0
part_1_2               0
rpt_dist_no            0
area_name              0
area                   0
time_occ               0
date_occ               0
status_desc            0
status                 0
crm_cd_1               0
location               0
lat                    0
lon                    0
dtype: int64

#### VICT_SEX

Usaremos el SimpleImputer para aplicar sobre la variable 'vict_sex' que vamos a sustituir los nulos por 'U' y sobre 'premis_cd' imputaremos por la moda

In [11]:
#['M', 'F', nan, 'X', 'H', '-']

#Cargamos e instanciamos SimpleImputer
from sklearn.impute import SimpleImputer

#Reenplaza todos los 'unknown' de df y los hace NaN de Numpy para que después los identifique bien.
#El valor a reemplazar hay que mirarlo en cada caso
df.vict_sex.replace(['nan'], np.nan, inplace = True)

#Instanciamos el nuevo imputer
#Variables CATEGÓRICAS
nulos_var = SimpleImputer(strategy='constant', fill_value='Unknwon') #Reenplazamos por un valor constante

#Entrenamos el imputer
#Es importante que si hacemos una sola variable la pongamos con doble corchete. Si no, nos dará un error de dimensiones
nulos_var.fit(df[['vict_sex']])

#Ejecutamos el imputer
df[['vict_sex']] = nulos_var.transform(df[['vict_sex']])

Eliminamos el registro de vict_sex = '-'

In [12]:
df = df.loc[~(df.vict_sex=='-')]

Estandarización de los valores de vict_sex para que solamente haya 'Male', 'Female' y 'Unknown'

In [13]:
df.vict_sex.unique()

array(['M', 'F', 'Unknwon', 'X', 'H'], dtype=object)

In [14]:
df.vict_sex.replace({'M':'Female',
                     'F':'Female',
                     'H':'Male',
                     'X':'Male'}, inplace=True)

In [15]:
df.vict_sex.unique()

array(['Female', 'Unknwon', 'Male'], dtype=object)

### VALORES ATÍPICOS

Localizamos los valores atípicos

In [16]:
for variable in df:
    print(variable + '\n')
    print(df[variable].value_counts(dropna = False))
    print('\n\n')

date_rptd

date_rptd
2023-01-03    315
2023-02-02    305
2022-12-02    302
2022-05-03    301
2023-02-03    300
             ... 
2025-02-28      1
2025-01-08      1
2025-02-10      1
2025-03-09      1
2025-03-01      1
Name: count, Length: 1861, dtype: int64



date_occ

date_occ
2022-12-01    377
2022-12-02    374
2020-01-01    364
2023-01-01    360
2022-10-01    358
             ... 
2025-01-31      1
2025-03-17      1
2025-02-19      1
2025-03-14      1
2025-01-17      1
Name: count, Length: 1853, dtype: int64



time_occ

time_occ
1200    11497
1800     8640
1700     8250
2000     8096
2200     7678
        ...  
436         8
531         7
854         7
151         6
424         6
Name: count, Length: 1439, dtype: int64



area

area
1     22831
12    20297
14    19796
3     18883
6     17343
15    16939
20    16590
18    16395
13    16245
7     15928
2     15431
8     15179
9     14260
11    14126
10    13892
17    13655
5     13591
21    13493
19    13338
4     12122
16    10926

#### AGRUPACION DE CATEGORÍAS RARAS

In [17]:
# Función de agrupacion de variables raras
def agrupar_cat_raras(variable, criterio = 0.05):
    #Calcula las frecuencias
    frecuencias = variable.value_counts(normalize=True)
    #Identifica las que están por debajo del criterio
    temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
    #Las recodifica en 'OTROS'
    temp2 = np.where(variable.isin(temp),'OTROS',variable)
    #Devuelve el resultado
    return(temp2)

# Variables a agrupar
var_imputar_cat_raras = ['weapon_used_cd','premis_cd','premis_desc','weapon_used_cd','weapon_desc','crm_cd_1','crm_cd_2','crm_cd_3','crm_cd_4','cross_street','location']

# Aplicación
for variable in var_imputar_cat_raras:
    df[variable] = agrupar_cat_raras(df[variable],criterio = 0.02)

Hacemos un describe sobre las variables numéricas a ver que vemos y observamos que, aunque hay varias variables numéricas, no podríamos hacer ningún estadísticos sobre ellas, exceptopsobre 'vict_age' donde vemos que la media de edad es de 29 años y la mediana está en 22 años pero observamos que el mínimo es -4 y máximo 120 años. Esto lo solucionaremos una media winsorizada (0,100)

In [18]:
df.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
time_occ,331260.0,1339.725841,651.545633,1.0000,900.0000,1420.0000,1900.0000,2359.000
area,331260.0,10.695620,6.103260,1.0000,5.0000,11.0000,16.0000,21.000
rpt_dist_no,331260.0,1116.009117,610.492137,101.0000,589.0000,1139.0000,1613.0000,2199.000
part_1_2,331260.0,1.400664,0.490034,1.0000,1.0000,1.0000,2.0000,2.000
crm_cd,331260.0,500.394572,205.419692,110.0000,331.0000,442.0000,626.0000,956.000
vict_age,331260.0,28.915891,22.009206,-4.0000,0.0000,30.0000,44.0000,99.000
lat,331260.0,33.996743,1.626516,0.0000,34.0148,34.0590,34.1649,34.333
lon,331260.0,-118.085676,5.637404,-118.6676,-118.4301,-118.3231,-118.2740,0.000


In [19]:
df.vict_age.value_counts().sort_index(ascending=True)

vict_age
-4         1
-3         2
-2        11
-1        31
 0     88752
       ...  
 95       35
 96       32
 97       22
 98       21
 99      109
Name: count, Length: 103, dtype: int64

#### WINSORIZACIÓN

Realizamos una imputacion de los valores de la edad tras realizar una media winsorizada manualmente (0, 100)

In [20]:
variable = 'vict_age'
df[variable].describe().loc[['min','max']]

min    -4.0
max    99.0
Name: vict_age, dtype: float64

In [21]:
minimo = 0
maximo = 100
df[variable] = df[variable].clip(minimo, maximo)
df[variable].describe()

count    331260.000000
mean         28.916081
std          22.008948
min           0.000000
25%           0.000000
50%          30.000000
75%          44.000000
max          99.000000
Name: vict_age, dtype: float64

In [22]:
num = df.select_dtypes(include='number')
num

,time_occ,area,rpt_dist_no,part_1_2,crm_cd,vict_age,lat,lon
dr_no,,,,,,,,
211204316,1718,12,1239,1,440,34,33.9883,-118.2915
201421864,1200,14,1445,2,354,42,33.9980,-118.4399
202110014,1815,21,2155,1,210,0,34.1811,-118.6059
200714602,1900,7,721,1,121,38,34.0735,-118.3777
201811375,130,18,1841,1,210,43,33.9348,-118.2826
...,...,...,...,...,...,...,...,...
240112436,800,1,153,1,330,35,34.0488,-118.2518
241314815,1522,13,1333,1,442,0,34.0174,-118.2621
240705529,1930,7,735,1,331,28,34.0649,-118.3440


In [23]:
cat = df.select_dtypes(exclude='number')
cat.head()

,date_rptd,date_occ,area_name,crm_cd_desc,mocodes,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street
dr_no,,,,,,,,,,,,,,,,,,,
211204316,2020-12-22,2020-12-21,77th Street,THEFT PLAIN - PETTY ($950 & UNDER),0344 1822,Female,H,OTROS,OTROS,nan,NaN,IC,Invest Cont,440.0,nan,nan,nan,OTROS,NaN
201421864,2020-12-12,2020-11-26,Pacific,THEFT OF IDENTITY,0377,Female,O,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",nan,NaN,IC,Invest Cont,354.0,nan,nan,nan,OTROS,NaN
202110014,2020-06-01,2020-06-01,Topanga,ROBBERY,1227 1309 0355 0353 0449 0344 1822,Female,W,101.0,STREET,OTROS,OTROS,IC,Invest Cont,210.0,nan,nan,nan,OTROS,NaN
200714602,2020-10-06,2020-10-03,Wilshire,"RAPE, FORCIBLE",1402 0500 0515 0913 1814,Female,W,101.0,STREET,OTROS,KNIFE WITH BLADE 6INCHES OR LESS,AO,Adult Other,OTROS,998.0,nan,nan,OTROS,OTROS
201811375,2020-05-20,2020-05-20,Southeast,ROBBERY,1822 1300 1310 0216 0342 0344 0408 0416 0445,Female,H,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,210.0,nan,nan,nan,OTROS,OTROS


## 4. GUARDADO DE RESULTADOS

In [24]:
ruta_principal = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/EVOLVE/Data Science/EVOLVE/Fernando_Costa/Practicas/Mini_Proyecto_EDA/'
carpeta = '002_archivos/'

In [25]:
ruta_trabajo = ruta_principal + carpeta + 'trabajo_resultado_eda.pickle'
ruta_cat = ruta_principal + carpeta + 'cat_resultado_eda.pickle'
ruta_num = ruta_principal + carpeta + 'num_resultado_eda.pickle'

Guardar los archivos

In [26]:
df.to_pickle(ruta_trabajo)
cat.to_pickle(ruta_cat)
num.to_pickle(ruta_num)